In [25]:
import pandas as pd
import time
import os

In [26]:
sectors = range(10, 65, 5)

In [27]:
#gvkey is unique identifier
df_dict = {'gvkey':[], 'predicted_return':[], 'trade_date':[]}

In [ ]:
start=time.time()
for sector in sectors:
    os.system(f"python3 fundamental_run_model.py -sector_name sector{sector} -tic_column gvkey -fundamental final_ratios.csv -sector sector{sector}.xlsx ")
    df = pd.read_csv(f"results/sector{sector}/df_predict_best.csv", index_col=0)
    for idx in df.index:
        predicted_return = df.loc[idx]
        top_q = predicted_return.quantile(0.75)
        predicted_return = predicted_return[predicted_return >= top_q]
        for gvkey in predicted_return.index:
            df_dict["gvkey"].append(gvkey)
            df_dict["predicted_return"].append(predicted_return[gvkey])
            df_dict["trade_date"].append(idx)
end=time.time()

In [6]:
print("it took ", (end-start)/60, ' minutes')

it took  0.40141645272572835  minutes


In [6]:
df_result = pd.DataFrame(df_dict)

In [7]:
df_result.to_csv("stock_selected.csv")

In [9]:
df_result.head()

,gvkey,predicted_return,trade_date
0,2991,-0.000871,2001-03-01
1,4430,0.001057,2001-03-01
2,7017,-0.001656,2001-03-01
3,7912,0.002009,2001-03-01
4,8068,-0.000118,2001-03-01
